# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


        
def draw_lines(img, lines):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    def draw_line(slopes, centers, first_point_y, second_point_y, color=[255, 0, 0], thickness=5):
        """
        draw individual lines for each lane
        """
        #average slope of all segments
        slope = np.sum(slopes)/len(slopes)
        #avarage center coord of all segments
        center = np.divide(np.sum(centers,axis=0), len(centers))
        
        def calc_x(point, y_coord, slope):
            """
            compute x coord of other point given y coord of this point, 
            line's slope, and other point on the line 
            """
            return point[0] + (y_coord - point[1]) / slope
                     
        first_point_x = calc_x(center, first_point_y, slope)       
        second_point_x = calc_x(center, second_point_y, slope)  
        
        #draw the lane line
        cv2.line(img, (int(first_point_x), int(first_point_y)), ( int(second_point_x), int(second_point_y) ), color, thickness)   
    
    #arrays for line segments' centers 
    rc = []
    lc = []
    
    #arrays for line segments' slopes
    rm = []
    lm = []
    
    imshape= img.shape
    imheight = imshape[0]
    imwidth = imshape[1]
    
    # this is the y coord of the highest point of extrapolated line
    upper_y = imheight*.6
    
    for line in lines:
        for x1,y1,x2,y2 in line:                        
            slope = (y2 - y1)/(x2-x1)
            center = ((x1+x2)/2, (y1+y2)/2)                
            
            # filtering line segments by slope into left lane, right lane and discarding others
            if .1 < slope < 10:             
                rm.append(slope)
                rc.append(center)
                segment_color = [0,255,0]
                
            elif -10 < slope < -.1:
                lm.append(slope)
                lc.append(center)
                segment_color = [0,0,255]
            else:
                segment_color = [255,255,255]
                
            segment_thickness = 5
            cv2.line(img,(x1,y1),(x2,y2), segment_color, segment_thickness)
        
    draw_line(rm, rc, imheight, upper_y)
    draw_line(lm, lc, imheight, upper_y)
       
    return img

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

In [ ]:
# Import everything needed to edit/save/watch video clips

from moviepy.editor import VideoFileClip
from IPython.display import HTML

## My pipeline

In [ ]:
def process_image(image):
    gray = grayscale(image)
    
    blur_gray = gaussian_blur(gray, 3)
    
    edges = canny(blur_gray, 50, 150)
    
    imshape=image.shape
    vertices = np.array([[
    (.51*imshape[1],imshape[0]*.58),
    (.49*imshape[1],imshape[0]*.58),
    (0,imshape[0]),
    (imshape[1],imshape[0])]], dtype = np.int32)
    
    target = region_of_interest(edges, vertices)
    
    lines = hough_lines(target, 1, np.pi/180, 35, 5, 2)
    
    result = weighted_img(lines, image, α=0.8, β=1.)
    
    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
import os
if not os.path.exists("test_videos_output"):
    os.makedirs("test_videos_output")
    
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))